In [1]:
from scipy.integrate import odeint
from scipy import *
import matplotlib.pyplot as plt
#from mpl_toolkits.mplot3d import Axes3D

import bqplot as bq
from ipywidgets import interact, interactive, Layout, Box, HBox, VBox, Label
import ipywidgets as ipyw
from IPython.display import display

%matplotlib inline

In [2]:
### Four state serca model class
class serca:
    
    ## Must be in same order as dXdt return
    label = ['x0', 'x1', 'x2', 'y0', 'y1', 'y2', 'ca_e']
    nVar = len(label)
    
    vol_er = (3.9*0.1*0.1) # all units in um
    vol_cyt = (4.0*0.5*0.5-vol_er)
    vol_tot = vol_cyt + vol_er
    Pt=15e-6
    gamma=vol_er/vol_cyt
    c=100e-9
    
    cai = 100.0e-9
    cae = 0
    nserca = 8678.0
    N_avo=6.022e23
    initial = array([0.4, 0.1, 0.0, 0.4, 0.1, 0.0,1])
    #initial = initial*nserca/(vol_er/1e18)/N_avo
    initial[6] = cae

    
    
    #eq_value=[Pt/4,Pt/4,gamma*Pt/4,gamma*Pt/4,250e-6]
    
    ## Parameters
    p_orig = {
    'kx0_x1':2*1.0e8,
    'kx1_x2':1.0e8,
    'kx1_x0':83.666,
    'kx2_x1':2*83.666,
    'kx2_y2':0.6,
    'ky2_x2':4.118,
    'ky2_y1':2*30.015,
    'ky1_y0':30.015,
    'ky1_y2':1e5,
    'ky0_y1':2e5,
    'ky0_x0':0.4,
    'kx0_y0':1.20e-3
    }
    p=p_orig.copy()

    ## Get initial values for the system
    def __init__(self, X0=initial, T=arange(0.0, 1e-3, 1e-6),f=1):
        self.T = T
        self.f=f
        self.name = self.__class__.__name__
        if self.nVar != len(X0):
            print 'ERROR:', self.nVar, 'initial values required for:', self.name
        else: 
            self.X0 = X0
            
    def dXdt1(self,X,t):
        x1,x2,y1,y2,ce=X
        for var in self.p.keys():
            exec(var+'='+str(self.p[var]))
        
        dx1 = k4*y1/self.gamma-k_4*x1-k1*self.c**2*x1+k_1*x2
        dx2 = k1*self.c**2*x1-k_1*x2+k_2*y2/self.gamma-k2*x2
        dy1 = k2*y2-k_3*ce**2*y1-k4
        dy2 = k2*x2/self.gamma-k_2*y2-k3*y2+k_3*ce**2*y1
        dce = 2*k3*y2-2*k_3*ce**2*y1
        return [dx1,dx2,dy1,dy2,dce]
    
    def dXdt(self,X,t):
        x0, x1, x2, y0, y1, y2, ca_e = X 
        
        for var in self.p.keys():
            exec(var+'='+str(self.p[var]))
            
        dx0 = x0*(-kx0_x1*self.cai-kx0_y0)+x1*kx1_x0+y0*ky0_x0
        dx1 = x1*(-kx1_x2*self.cai-kx1_x0)+x0*self.cai*kx0_x1+x2*kx2_x1
        dx2 = x2*(-kx2_y2-kx2_x1)+x1*self.cai*kx1_x2+y2*ky2_x2
        dy0 = y0*(-ky0_y1*ca_e-ky0_x0)+y1*ky1_y0+x0*kx0_y0
        dy1 = y1*(-ky1_y2*ca_e-ky1_y0)+y0*ca_e*ky0_y1+y2*ky2_y1
        dy2 = y2*(-ky2_x2-ky2_y1)+y1*ca_e*ky1_y2+x2*kx2_y2
        dcae = -ca_e*(y1*ky1_y2 + y0*ky0_y1) + (y1*ky1_y0 + y2*ky2_y1)
        return [dx0, dx1, dx2, dy0, dy1, dy2, dcae]
    
    def update_p(self,m=1,to_update=p.keys()):
        for k in to_update:
            self.p[k]=m*self.p_orig[k]
            #print self.p

In [3]:
class slider():
    cpalette = ['#EE224A', '#22AF4B', '#4CB5F5', '#FF5C00', 
                '#08B9A5', '#15AB00', '#881EE4', '#5C6BC0']
    
    def __init__(self, model):
        self.model = model
        self.lines = {}
        self.param = {}
        self.cb = {}
        self.colors = {}
        
        self._solve()
        self._curveColors()
        self._plot()
        
        
    def _curveColors(self):
        for i,v in enumerate(self.model.label):
            self.colors.update({v: self.cpalette[i]})
        
    def _solve(self):
        sol = odeint(self.model.dXdt, self.model.X0, self.model.T)
        
        self.sol = {}
        for i,var in enumerate(self.model.label):
            self.sol.update({var: sol[:,i]})
    
    def _getFig(self):
        for v in self.model.label:
            x_sc = bq.LinearScale()
            y_sc = bq.LinearScale()

            x_ax = bq.Axis(label='Time', scale=x_sc, tick_format=None, color='black', grid_lines='solid', grid_color='#ddd')
            y_ax = bq.Axis(label=v,      scale=y_sc, tick_format=None, color='black', grid_lines='solid', grid_color='#ddd', orientation='vertical')

            # Generate a line (but does not plot it)
            l = bq.Lines(x=self.model.T, y=self.sol[v], colors=[self.colors[v]],
                              scales={'x': x_sc, 'y': y_sc})
        
            self.lines.update({v: l})
          
        fig = bq.Figure(axes=[x_ax, y_ax], marks=self.lines.values(),
                       fig_margin={'top':10, 'bottom':0, 'left':60, 'right':10},
                       max_aspect_ratio=3, min_aspect_ratio=2.5)
        fig.layout.width = '100%'
        return fig
        
    def _plot(self):
        self._solve()
        fig = self._getFig()
        
        display(fig)
        
    def _updateFig(self):        
        self._solve()
        for v in self.model.label:
            self.lines[v].y = self.sol[v]
    
    ### Parameter handling
    def _paramUpdate(self, change):
        self.model.p[change['owner'].description] = change['new']
        self._updateFig()
            
    def paramSlider(self):
        for k,v in self.model.p.items():
            crude = ipyw.IntSlider(
                min = -5, max = 10,
                description=k,
                value = floor(log10(v)), 
                continuous_update = False
            )

            fine = ipyw.FloatSlider(
                min = 10**floor(log10(v)),
                max = 10**(floor(log10(v))+1),
                description=k, 
                value = v,
                continuous_update = False
            )
            
            #txt = ipyw.FloatText(value=v, description=k, layout=Layout(width='80px'))
        
            #ipyw.jslink((fine, 'value'), (txt, 'value'))
            crude.observe(self._updateRange, names='value')
            fine.observe(self._paramUpdate,  names='value')
        
            p = [crude, fine]#, txt]
            self.param.update({k: p})
        
        box_layout = Layout(display='flex-start',
                    flex_flow='row',
                    align_items='flex-start',
                    align_content='flex-start',
                    width='100%')
        
        for k, v in self.param.items():
            box = Box(children=v, layout=box_layout)
            display(box, layout=Layout(align_items='flex-start'))
            
            
    def _updateRange(self, c):
        k = c['owner'].description
        try:
            self.param[k][1].max = 10**(c['new']+1)
            self.param[k][1].min = 10**c['new']
        except:
            self.param[k][1].min = 10**c['new']
            self.param[k][1].max = 10**(c['new']+1)
        self.param[k][1].value = self.param[k][1].min
        self.param[k][1].step = 1 if c['new']>0 else 10**(c['new']-1)
        
    ### Toggle curve display
    def _toggleCurve(self, b):
        v = not self.lines[b.description].visible
        if self.lines[b.description].visible:
            self.cb[b.description].style.button_color = '#ddd'
            self.lines[b.description].visible = v
        else:
            self.cb[b.description].style.button_color = self.colors[b.description]
            self.lines[b.description].visible = v
        
        
    def _getToggleButton(self, v):
        cb = ipyw.Button(
            value = self.lines[v].visible,
            description = v,
            style = ipyw.ButtonStyle(button_color=self.colors[v]),
            layout = Layout(width='100px')
        )
        return cb
        
    def toggleButton(self):
        for v in self.model.label:
            self.cb.update({v: self._getToggleButton(v)})
            
        display(VBox(self.cb.values()))
        
        for cb in self.cb.values():
            cb.on_click(self._toggleCurve)

In [4]:
T = arange(0.0, 100, 1e-1)
S=serca(T=T)
#S.f=1e3
#S.update_p()
m = slider(model=S)
#S.p_orig
m.paramSlider()
m.toggleButton()

RmlndXJlKGF4ZXM9W0F4aXMoY29sb3I9J2JsYWNrJywgZ3JpZF9jb2xvcj0nI2RkZCcsIGxhYmVsPXUnVGltZScsIHNjYWxlPUxpbmVhclNjYWxlKCkpLCBBeGlzKGNvbG9yPSdibGFjaycsIGfigKY=


Qm94KGNoaWxkcmVuPShJbnRTbGlkZXIodmFsdWU9OCwgY29udGludW91c191cGRhdGU9RmFsc2UsIGRlc2NyaXB0aW9uPXUna3gwX3gxJywgbWF4PTEwLCBtaW49LTUpLCBGbG9hdFNsaWRlcijigKY=


Qm94KGNoaWxkcmVuPShJbnRTbGlkZXIodmFsdWU9NSwgY29udGludW91c191cGRhdGU9RmFsc2UsIGRlc2NyaXB0aW9uPXUna3kwX3kxJywgbWF4PTEwLCBtaW49LTUpLCBGbG9hdFNsaWRlcijigKY=


Qm94KGNoaWxkcmVuPShJbnRTbGlkZXIodmFsdWU9LTEsIGNvbnRpbnVvdXNfdXBkYXRlPUZhbHNlLCBkZXNjcmlwdGlvbj11J2t5MF94MCcsIG1heD0xMCwgbWluPS01KSwgRmxvYXRTbGlkZXLigKY=


Qm94KGNoaWxkcmVuPShJbnRTbGlkZXIodmFsdWU9MSwgY29udGludW91c191cGRhdGU9RmFsc2UsIGRlc2NyaXB0aW9uPXUna3kyX3kxJywgbWF4PTEwLCBtaW49LTUpLCBGbG9hdFNsaWRlcijigKY=


...

Qm94KGNoaWxkcmVuPShJbnRTbGlkZXIodmFsdWU9MSwgY29udGludW91c191cGRhdGU9RmFsc2UsIGRlc2NyaXB0aW9uPXUna3gxX3gwJywgbWF4PTEwLCBtaW49LTUpLCBGbG9hdFNsaWRlcijigKY=


Qm94KGNoaWxkcmVuPShJbnRTbGlkZXIodmFsdWU9MSwgY29udGludW91c191cGRhdGU9RmFsc2UsIGRlc2NyaXB0aW9uPXUna3kxX3kwJywgbWF4PTEwLCBtaW49LTUpLCBGbG9hdFNsaWRlcijigKY=


Qm94KGNoaWxkcmVuPShJbnRTbGlkZXIodmFsdWU9MiwgY29udGludW91c191cGRhdGU9RmFsc2UsIGRlc2NyaXB0aW9uPXUna3gyX3gxJywgbWF4PTEwLCBtaW49LTUpLCBGbG9hdFNsaWRlcijigKY=


Qm94KGNoaWxkcmVuPShJbnRTbGlkZXIodmFsdWU9MCwgY29udGludW91c191cGRhdGU9RmFsc2UsIGRlc2NyaXB0aW9uPXUna3kyX3gyJywgbWF4PTEwLCBtaW49LTUpLCBGbG9hdFNsaWRlcijigKY=


VkJveChjaGlsZHJlbj0oQnV0dG9uKGRlc2NyaXB0aW9uPXUneTEnLCBsYXlvdXQ9TGF5b3V0KHdpZHRoPXUnMTAwcHgnKSwgc3R5bGU9QnV0dG9uU3R5bGUoYnV0dG9uX2NvbG9yPScjMDhCOUHigKY=
